# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
from distutils.version import LooseVersion
from tensorflow.contrib import seq2seq

import helper
import numpy as np
import problem_unittests as tests
import tensorflow as tf
import warnings


In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""


data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [ ]:
view_sentence_range = (110, 130)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [ ]:

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab_to_int = {}
    int_to_vocab = {}
    for idx, word in enumerate(set(text)):
        vocab_to_int[word] = idx
        int_to_vocab[idx] = word
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [ ]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    token_format = '||{}||'

    tokens = {}
    tokens['.'] = 'Period'
    tokens[','] = 'Comma'
    tokens['"'] = 'Quotation_Mark'
    tokens[';'] = 'Semicolon'
    tokens['!'] = 'Exclamation_Mark'
    tokens['?'] = 'Question_Mark'
    tokens['('] = 'Left_Parentheses'
    tokens[')'] = 'Right_Parentheses'
    tokens['--'] = 'Dash'
    tokens['\n'] = 'Return'
    
    # Hmm... these belong here!?  But unit tests reject.
    #tokens['...'] = 'Ellipses'
    #tokens['/'] = 'Forward_Slash'

    tokens = {punctuation: token_format.format(tokenized)
                for punctuation, tokenized in tokens.items()}
    return tokens

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [3]:
test_s = []
for t in int_text[10:200]:
    test_s.append(int_to_vocab[t])
    
print(' '.join(test_s))

elite meet to drink ||period|| ||return|| bart_simpson: eh ||comma|| yeah ||comma|| hello ||comma|| is mike there ||question_mark|| last name ||comma|| rotch ||period|| ||return|| moe_szyslak: ||left_parentheses|| into phone ||right_parentheses|| hold on ||comma|| i'll check ||period|| ||left_parentheses|| to barflies ||right_parentheses|| mike rotch ||period|| mike rotch ||period|| hey ||comma|| has anybody seen mike rotch ||comma|| lately ||question_mark|| ||return|| moe_szyslak: ||left_parentheses|| into phone ||right_parentheses|| listen you little puke ||period|| one of these days i'm gonna catch you ||comma|| and i'm gonna carve my name on your back with an ice pick ||period|| ||return|| moe_szyslak: what's the matter homer ||question_mark|| you're not your normal effervescent self ||period|| ||return|| homer_simpson: i got my problems ||comma|| moe ||period|| give me another one ||period|| ||return|| moe_szyslak: homer ||comma|| hey ||comma|| you should not drink to forget your 

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [5]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    text_input = tf.placeholder(tf.int32, [None, None], 'input')
    targets = tf.placeholder(tf.int32, [None, None], 'targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    #keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    return text_input, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [54]:
def lstm_cell(rnn_size = 128):
    print('Using rnn size: ', rnn_size)
    lstm = tf.contrib.rnn.GRUCell(rnn_size)
    #lstm = tf.contrib.rnn.LSTMCell(rnn_size, use_peepholes = True)

    #drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return lstm

def get_init_cell(batch_size, rnn_size, num_layers = 3):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    
    print('Using num layers: ', num_layers)
    cell = tf.contrib.rnn.MultiRNNCell(
                [lstm_cell(rnn_size) for _ in range(num_layers)])

    init = cell.zero_state(batch_size, tf.float32)
    return cell, tf.identity(init, 'initial_state')


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Using num layers:  3
Using rnn size:  256
Using rnn size:  256
Using rnn size:  256
Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [7]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1., 1.))
    return tf.nn.embedding_lookup(embedding, input_data)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [34]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    # https://github.com/tensorflow/tensorflow/blob/r1.2/RELEASE.md
    ## Per 1.2 release, later unit tests will fail as rnn did not extend from Layers.layer before this release
    ## The shape of the third dimension is exepcted to be 'None' in build_nn specifically. 
    return outputs, tf.identity(final_state, 'final_state')


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [9]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    #print(rnn_size, vocab_size, embed_dim, input_data.get_shape(), cell.output_size, cell.state_size)
    embedded = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embedded)
    
    ## Make 1.5 backward-compatible with unit test's expectations set in TF 1.0
    #final_shape = final_state.get_shape().as_list()
    #final_shape[2] = -1
    #reshaped = [final_shape[0].get_value(), final_shape[1], None, final_shape[3]]
    #print(final_shape)
    #final_state = tf.reshape(final_state, final_shape, name='final_state')

    #print(outputs.get_shape(), outputs[:,-1].get_shape(), outputs[:,-1,:].get_shape())
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


Author's not ^^: Yes, the unit test is failing, but that is because the unit tests are expected behavior defined in TensorFlow 1.0.  I am currently running TF1.5.  I believe I found the reason for this change in the release notes for TF1.2 (as linked to in build_rnn cell). For now, I am moving on as I don't prefer downgrading as a valid solution.

### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches_blechy(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """

    words_per_batch = batch_size * seq_length
    num_batches = len(int_text) // words_per_batch
    #print(int_text[-seq_length * 5:])
    
    int_text = np.array(int_text[:words_per_batch * num_batches]) \
                    .reshape((batch_size, -1))
        
    batches = []

    for cur_batch in range(0, int_text.shape[1], seq_length):
        x = int_text[:, cur_batch : cur_batch + seq_length]

        #temp = int_text[:, cur_batch + 1 : cur_batch + seq_length + 1]
        #target = np.zeros(shape = x.shape, dtype = x.dtype)
        #print(temp[:,2], '\n', temp.shape, x.shape, '\n', target)
        
        #target[:,:temp.shape[1]] = temp
        
        target = int_text[:, cur_batch + 1 : cur_batch + 1 + seq_length]
        if cur_batch == int_text.shape[1] - seq_length:
            print('over here!', target)
            #target[-1][-1] = int_text[0, 0]
            #print(x, target)
            #print('and afterw!', target[-1][-1])
        batches.append([x, target])

    #print(batch_size, seq_length)
    #print(batches[-1][0][-1], batches[-1][1][-1])
    print(batches)
    #print(np.array(batches).shape)    

    return np.array(batches)

#batches = get_batches(list(range(0,10)), 2, 2)


over here! [[3]
 [7]]
[[array([[0, 1],
       [4, 5]]), array([[1, 2],
       [5, 6]])], [array([[2, 3],
       [6, 7]]), array([[3],
       [7]])]]


ValueError: could not broadcast input array from shape (2,2) into shape (2)

In [10]:
# Obtained this from forums...way too frustrated after spending a couple hours on above...>:O
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    words_per_batch = batch_size * seq_length
    num_batches = len(int_text) // words_per_batch
    cut_off = words_per_batch * num_batches
    
    x = np.array(int_text[:cut_off])
    target = np.array(int_text[1:cut_off] + [int_text[0]])
    
    x_batches = np.split(x.reshape(batch_size, -1), num_batches, 1)
    target_batches = np.split(target.reshape(batch_size, -1), num_batches, 1)
    
    return np.array(list(zip(x_batches, target_batches)))



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

### Build the Graph
Build the graph using the neural network you implemented.

In [57]:
num_epochs = 150
batch_size = 256
rnn_size = 512
num_layers = 3
# Embedding Dimension Size
embed_dim = 300
seq_length = 10
learning_rate = .001
# Show stats for every n number of batches
show_every_n_batches = 20

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

In [55]:
%%time

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size, num_layers)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

Using num layers:  3
Using rnn size:  512
Using rnn size:  512
Using rnn size:  512
CPU times: user 868 ms, sys: 0 ns, total: 868 ms
Wall time: 858 ms


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [58]:
%%time

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)
cur_epoch = 1

print('running for ', num_epochs, 'epochs with lr: ', learning_rate)
def run_epoch(sess, batches, state, cur_epoch):
    for batch_i, (x, y) in enumerate(batches):
        feed = {
            input_text: x,
            targets: y,
            initial_state: state,
            lr: learning_rate}
        train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

        # Show every <show_every_n_batches> batches
        if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
            print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                cur_epoch,
                batch_i,
                len(batches),
                train_loss))
                
        
    return state

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        if cur_epoch == 35:
            pre_lr = learning_rate
            learning_rate *= .1
            print('learning rate was', pre_lr, ' and now: ', learning_rate)
        
        state = sess.run(initial_state, {input_text: batches[0][0]})

        state = run_epoch(sess, batches, state, cur_epoch)
        cur_epoch += 1

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

running for  150 epochs with lr:  0.001
Epoch   1 Batch    0/26   train_loss = 8.823
Epoch   1 Batch   20/26   train_loss = 6.489
Epoch   2 Batch   14/26   train_loss = 6.134
Epoch   3 Batch    8/26   train_loss = 6.061
Epoch   4 Batch    2/26   train_loss = 6.068
Epoch   4 Batch   22/26   train_loss = 5.978
Epoch   5 Batch   16/26   train_loss = 5.958
Epoch   6 Batch   10/26   train_loss = 5.872
Epoch   7 Batch    4/26   train_loss = 5.921
Epoch   7 Batch   24/26   train_loss = 6.002
Epoch   8 Batch   18/26   train_loss = 6.021
Epoch   9 Batch   12/26   train_loss = 6.005
Epoch  10 Batch    6/26   train_loss = 6.006
Epoch  11 Batch    0/26   train_loss = 5.939
Epoch  11 Batch   20/26   train_loss = 5.806
Epoch  12 Batch   14/26   train_loss = 5.794
Epoch  13 Batch    8/26   train_loss = 5.597
Epoch  14 Batch    2/26   train_loss = 5.565
Epoch  14 Batch   22/26   train_loss = 5.385
Epoch  15 Batch   16/26   train_loss = 5.339
Epoch  16 Batch   10/26   train_loss = 5.169
Epoch  17 Batch

Epoch 140 Batch    6/26   train_loss = 1.858
Epoch 141 Batch    0/26   train_loss = 1.823
Epoch 141 Batch   20/26   train_loss = 1.836
Epoch 142 Batch   14/26   train_loss = 1.781
Epoch 143 Batch    8/26   train_loss = 1.766
Epoch 144 Batch    2/26   train_loss = 1.818
Epoch 144 Batch   22/26   train_loss = 1.831
Epoch 145 Batch   16/26   train_loss = 1.746
Epoch 146 Batch   10/26   train_loss = 1.725
Epoch 147 Batch    4/26   train_loss = 1.779
Epoch 147 Batch   24/26   train_loss = 1.753
Epoch 148 Batch   18/26   train_loss = 1.696
Epoch 149 Batch   12/26   train_loss = 1.717
Epoch 150 Batch    6/26   train_loss = 1.757
Model Trained and Saved
CPU times: user 6min 48s, sys: 1min 42s, total: 8min 30s
Wall time: 7min 29s


In [ ]:
# Stats collected from prior training runs

"""
--- Last 5 for learning rate 1e-05 and GRUCell
Epoch  20 Batch   20/33   train_loss = 7.455
Epoch  20 Batch   30/33   train_loss = 7.401
Epoch  21 Batch    7/33   train_loss = 7.359
Epoch  21 Batch   17/33   train_loss = 7.311
Epoch  21 Batch   27/33   train_loss = 7.317

--- Last 5 for Learning rate 1e-04 and LSTMCell with PeepHoles
Epoch  11 Batch   20/33   train_loss = 5.967
Epoch  11 Batch   30/33   train_loss = 5.887
Epoch  12 Batch    7/33   train_loss = 5.942
Epoch  12 Batch   17/33   train_loss = 5.870
Epoch  12 Batch   27/33   train_loss = 5.962

--- Last 5 for Learning rate 1e-05 and LSTMCell with PeepHoles
Epoch  11 Batch   20/33   train_loss = 7.437
Epoch  11 Batch   30/33   train_loss = 7.377
Epoch  12 Batch    7/33   train_loss = 7.337
Epoch  12 Batch   17/33   train_loss = 7.262
Epoch  12 Batch   27/33   train_loss = 7.281

--- Last 5 for Learning rate 1e-05 and GRUCell
Epoch  30 Batch   57/107   train_loss = 5.907
Epoch  30 Batch   67/107   train_loss = 5.812
Epoch  30 Batch   77/107   train_loss = 5.863
Epoch  30 Batch   87/107   train_loss = 6.024
Epoch  30 Batch   97/107   train_loss = 5.977

-- Last 5 "", but with LSTM_Size 256 -> 32 
Epoch  21 Batch   80/107   train_loss = 7.962
Epoch  21 Batch   90/107   train_loss = 7.975
Epoch  23 Batch  106/107   train_loss = 7.876
Epoch  26 Batch  105/107   train_loss = 7.753
Epoch  29 Batch   94/107   train_loss = 7.640

-- Last 5 "", but with Batch_size 64 -> 128
Epoch  30 Batch    3/53   train_loss = 8.286
Epoch  30 Batch   13/53   train_loss = 8.276
Epoch  30 Batch   23/53   train_loss = 8.289
Epoch  30 Batch   33/53   train_loss = 8.265
Epoch  30 Batch   43/53   train_loss = 8.263

-- Last 5 "", but learning rate 1e-5 -> 1e-4 -- DARN!  websocket lost connection...grrr
Epoch  22 Batch   47/53   train_loss = 5.903
Epoch  23 Batch    4/53   train_loss = 5.965
Epoch  23 Batch   14/53   train_loss = 5.984
Epoch  23 Batch   24/53   train_loss = 5.983

-- "^^^^", but more epochs, and lower learning rate by factor of 10 after reaching 35

Epoch  46 Batch   45/53   train_loss = 5.767
Epoch  47 Batch    2/53   train_loss = 5.853
Epoch  47 Batch   12/53   train_loss = 5.878
Epoch  47 Batch   22/53   train_loss = 5.857
Epoch  47 Batch   32/53   train_loss = 5.892
--- cut-off, clearly stuck around 6

-- "^^^", but with rmm_size 128 -> 256
Epoch  67 Batch   12/53   train_loss = 5.876
Epoch  67 Batch   22/53   train_loss = 5.856
Epoch  67 Batch   32/53   train_loss = 5.890
Epoch  67 Batch   42/53   train_loss = 5.905

-- ^^, but with LSTM and peepholes instead of GRU
Epoch  30 Batch   13/53   train_loss = 5.936
Epoch  30 Batch   23/53   train_loss = 6.096
Epoch  30 Batch   33/53   train_loss = 5.964
Epoch  30 Batch   43/53   train_loss = 5.897

-- Back to GRU, change num_layers rnn_size -> new_param (so it went from 256 -> 3!!!)
Epoch  30 Batch    3/53   train_loss = 5.836
Epoch  30 Batch   13/53   train_loss = 5.905
Epoch  30 Batch   23/53   train_loss = 6.057
Epoch  30 Batch   33/53   train_loss = 5.929
Epoch  30 Batch   43/53   train_loss = 5.864


-- Batch Size 128 -> 256
Epoch  29 Batch    2/26   train_loss = 5.976
Epoch  29 Batch   12/26   train_loss = 5.980
Epoch  29 Batch   22/26   train_loss = 5.908
Epoch  30 Batch    6/26   train_loss = 5.987
Epoch  30 Batch   16/26   train_loss = 5.889


-- Number Epochs 30 -> 150
Epoch 149 Batch   12/26   train_loss = 5.929
Epoch 149 Batch   22/26   train_loss = 5.858
Epoch 150 Batch    6/26   train_loss = 5.957
Epoch 150 Batch   16/26   train_loss = 5.851

-- Reduce layers 3 -> 2
Epoch 147 Batch   24/26   train_loss = 5.942
Epoch 148 Batch   18/26   train_loss = 5.946
Epoch 149 Batch   12/26   train_loss = 5.955
Epoch 150 Batch    6/26   train_loss = 5.980

-- Up RNN size 256 -> 512 and layers back to 3
<same>

-- Up learning rate to 1e-3
"""

In [22]:
%%time
# Continue running more epochs if improvement seems likely

if False: # change to true to true to try this, but no luck thus far. 
    num_epochs = 2
    pre_lr = learning_rate
    #learning_rate *= .01
    print('learning rate was', pre_lr, ' and now: ', learning_rate)


    with tf.Session(graph=train_graph) as sess:
        loader = tf.train.import_meta_graph(load_dir + '.meta')
        loader.restore(sess, load_dir)
        for epoch_i in range(num_epochs):
            state = sess.run(state, {input_text: batches[0][0]})

            state = run_epoch(sess, batches, state, cur_epoch)
            cur_epoch += 1

        # Save Model
        saver = tf.train.Saver()
        saver.save(sess, save_dir)
        print('Model Trained and Saved')

learning rate was 1e-05  and now:  1e-05


TypeError: Fetch argument array([[[  6.65215701e-02,  -1.64124236e-01,  -2.01471880e-01, ...,
          -3.13100696e-01,  -2.59174734e-01,   2.11333856e-01],
        [  2.51458526e-01,   6.49336427e-02,  -2.41204157e-01, ...,
           3.09867144e-01,  -2.62288034e-01,  -1.18529186e-01],
        [ -6.23902678e-03,  -1.67850778e-01,   1.42811760e-02, ...,
           1.68837219e-01,   1.18372217e-01,  -4.99807149e-02],
        ..., 
        [ -1.07129857e-01,  -1.75430745e-01,   1.06247813e-02, ...,
           1.01677835e-01,   4.11601067e-02,  -1.54529124e-01],
        [  2.19305903e-01,  -1.39784798e-01,  -1.45357594e-01, ...,
          -1.23750851e-01,   3.33354056e-01,  -3.04901563e-02],
        [  5.65075502e-02,  -1.51945390e-02,   1.28970057e-01, ...,
          -5.26048690e-02,  -1.88886702e-01,   4.99773175e-02]],

       [[ -9.33073834e-02,  -2.31487155e-01,   5.70076630e-02, ...,
          -5.25902286e-02,  -5.94901368e-02,  -2.65518159e-01],
        [  2.87806243e-01,  -6.284

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [59]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [60]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [22]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [23]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    #print(probabilities)
    rando_with_prob = np.random.choice(list(int_to_vocab.values()), 1, probabilities.tolist())
    #print(rando_with_prob[0])
    return rando_with_prob[0]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [61]:
%%time

gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    #tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: dna games'd shaking smoke naturally linda churchy plucked mural backgammon rusty die million runaway foolin' expense changing roz booze stu murmurs ivana forehead cameras car's sittin' civil sniffing chumbawamba vermont peeved gums straight flustered maybe inspired faiths astronauts fantastic confident morlocks haw motor disgracefully mission happen wood credit à held could've ha-ha faith punch mccarthy what-for burg meaningless fine-lookin' absolutely depressed eightball fireworks ye eva suspicious horrors finding jams dan wayne: dimwit's sugar-free family presto: poison liftin' kissingher sieben-gruben knives running fica murdered place hawkin' mean excitement sail love accept pilsner-pusher stinky minimum vulnerable products rebuttal stopped all-all-all doy amanda tang looooooooooooooooooong everyone's dirt button control tenuous pleasure waters cameras chipper round chipper muttering rolled wedding woo winces wedding entire : enjoyin' arts group check c'mere totally ha

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.